In [1]:
import numpy as np
import pandas as pd

In [16]:
df_hs = pd.read_csv('Resources/hawaii_stations.csv')
df_hm = pd.read_csv('Resources/hawaii_measurements.csv')
df_hm = df_hm.dropna(axis=0,how='any')
df_hs.to_csv('clean_hawaii_stations.csv',index=False)
df_hm.to_csv('clean_hawaii_measurements.csv',index=False)